# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

Group Number: 35

Group Members:
- Angelica Aliwinata (18222113)
- Jason Jahja (18222116)
- Melissa Trenggono (18222123)
- Anindita Widya Santoso (18222128)

## Import Libraries

In [62]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import pickle

# Import tools for data counting
from collections import Counter

# Import tools for data splitting
from sklearn.model_selection import train_test_split

# Import tools for imputing data
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

# Import tools for oversampling
from imblearn.over_sampling import SMOTE

# Import tools for preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Import tools for pipeline
from sklearn.pipeline import Pipeline

# Import tools for testing accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.datasets import make_classification

from collections import Counter

## Import Dataset

In [33]:
# Reading CSV files from a gdrive link {Google Collab}
train_df = pd.read_csv('../data/train_download.csv')

# Display the first few rows of the DataFrame
train_df.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


### Summary of the Dataset

In [34]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140404 entries, 0 to 140403
Data columns (total 56 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          140404 non-null  int64  
 1   FILENAME                    82872 non-null   object 
 2   URL                         96917 non-null   object 
 3   URLLength                   79765 non-null   float64
 4   Domain                      70207 non-null   object 
 5   DomainLength                94085 non-null   float64
 6   IsDomainIP                  98274 non-null   float64
 7   TLD                         95005 non-null   object 
 8   CharContinuationRate        92362 non-null   float64
 9   TLDLegitimateProb           87531 non-null   float64
 10  URLCharProb                 88333 non-null   float64
 11  TLDLength                   92673 non-null   float64
 12  NoOfSubDomain               96344 non-null   float64
 13  HasObfuscation

### Statistics for All Features

In [35]:
train_df.describe(include='all')

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
count,140404.000000,82872,96917,79765.000000,70207,94085.000000,98274.000000,95005,92362.000000,87531.000000,...,97230.000000,90207.000000,73059.000000,89932.000000,73270.000000,79603.000000,92272.000000,97718.000000,71025.000000,140404.000000
unique,NaN,82872,96914,NaN,69832,NaN,NaN,497,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,8135291.txt,http://www.strangled.net,NaN,ipfs.io,NaN,NaN,com,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,2,NaN,66,NaN,NaN,47878,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,117682.632746,NaN,NaN,27.701473,NaN,19.619387,0.000488,NaN,0.918007,0.277482,...,0.345706,0.034454,0.751543,41.647489,10.268623,16.461729,104.875900,3.733846,78.104273,0.924831
std,68122.005080,NaN,NaN,23.140676,NaN,5.833808,0.022095,NaN,0.159142,0.248115,...,0.475600,0.182393,0.432121,102.538702,133.321659,21.001274,219.137035,24.192458,187.106965,0.263664
min,1.000000,NaN,NaN,14.000000,NaN,4.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58689.500000,NaN,NaN,23.000000,NaN,16.000000,0.000000,NaN,0.913043,0.012927,...,0.000000,0.000000,1.000000,10.000000,2.000000,6.000000,24.000000,0.000000,13.000000,1.000000
50%,117421.500000,NaN,NaN,26.000000,NaN,19.000000,0.000000,NaN,1.000000,0.522907,...,0.000000,0.000000,1.000000,23.000000,5.000000,12.000000,69.000000,0.000000,39.000000,1.000000
75%,176724.250000,NaN,NaN,30.000000,NaN,23.000000,0.000000,NaN,1.000000,0.522907,...,1.000000,0.000000,1.000000,45.000000,12.000000,22.000000,132.000000,3.000000,99.000000,1.000000


### Statistics for Numerical Features

In [36]:
train_df.describe()

,id,URLLength,DomainLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
count,140404.000000,79765.000000,94085.000000,98274.000000,92362.000000,87531.000000,88333.000000,92673.000000,96344.000000,74684.000000,...,97230.000000,90207.000000,73059.000000,89932.000000,73270.000000,79603.000000,92272.000000,97718.000000,71025.000000,140404.000000
mean,117682.632746,27.701473,19.619387,0.000488,0.918007,0.277482,0.059286,2.729015,1.161442,0.000402,...,0.345706,0.034454,0.751543,41.647489,10.268623,16.461729,104.875900,3.733846,78.104273,0.924831
std,68122.005080,23.140676,5.833808,0.022095,0.159142,0.248115,0.008063,0.533310,0.445054,0.020038,...,0.475600,0.182393,0.432121,102.538702,133.321659,21.001274,219.137035,24.192458,187.106965,0.263664
min,1.000000,14.000000,4.000000,0.000000,0.000000,0.000000,0.001229,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58689.500000,23.000000,16.000000,0.000000,0.913043,0.012927,0.055542,2.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,10.000000,2.000000,6.000000,24.000000,0.000000,13.000000,1.000000
50%,117421.500000,26.000000,19.000000,0.000000,1.000000,0.522907,0.060264,3.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,23.000000,5.000000,12.000000,69.000000,0.000000,39.000000,1.000000
75%,176724.250000,30.000000,23.000000,0.000000,1.000000,0.522907,0.064391,3.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,45.000000,12.000000,22.000000,132.000000,3.000000,99.000000,1.000000
max,235795.000000,4054.000000,93.000000,1.000000,1.000000,0.522907,0.088766,13.000000,7.000000,1.000000,...,1.000000,1.000000,1.000000,8956.000000,35820.000000,2828.000000,26596.000000,4887.000000,27516.000000,1.000000


In [37]:
columns_to_drop = ['FILENAME', 'Domain', 'URL', 'Title']
print("Columns before dropping columns:", train_df.shape[1])
train_df = train_df.drop(columns=columns_to_drop)
print("Columns after dropping columns:", train_df.shape[1])

Columns before dropping columns: 56
Columns after dropping columns: 52


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [38]:
# Keep original training set
train_set_copy = train_df.copy()

# Splitting training and validation set
train_df, validate_set = train_test_split(train_df, test_size=0.2, random_state=42)

# Check training and validation set size
print("Training Set:", train_df.shape)
print("Validation Set:", validate_set.shape)

Training Set: (112323, 52)
Validation Set: (28081, 52)


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [39]:
# Split numerical and non-numerical columns on training set
numerical_columns = train_df.select_dtypes(include=['float64', 'int64'])
categorical_columns = train_df.select_dtypes(include=['object'])

# Split numerical and non-numerical columns on validation set
numerical_columns = validate_set.select_dtypes(include=['float64', 'int64'])
categorical_columns = validate_set.select_dtypes(include=['object'])

In [40]:
# Calculate missing values
def missing_vals(column, dataset):
    
    # Count total missing values in the column
    total_missing = column.isnull().sum()
    
    # Filter columns with missing values and sort by descending order
    columns_with_missing = total_missing[total_missing > 0].sort_values(ascending=False)
    
    # Loop through each column with missing values and calculate percentage
    for column_name in columns_with_missing.index:
        missing_count = len(dataset) - dataset[column_name].value_counts().sum()
        missing_percentage = round((missing_count / len(dataset)) * 100, 2)
        print(f"{column_name} has {missing_percentage}% missing values ({missing_count} rows missing).")


#### Training Data

In [41]:
# Missing values on numerical columns
print("Missing numerical data on training set: ")
missing_vals(numerical_columns,train_df)

Missing numerical data on training set: 
LineOfCode has 49.18% missing values (55243 rows missing).
NoOfExternalRef has 49.41% missing values (55498 rows missing).
LargestLineLength has 48.38% missing values (54339 rows missing).
NoOfCSS has 47.67% missing values (53549 rows missing).
HasSocialNet has 48.5% missing values (54473 rows missing).
HasCopyrightInfo has 47.99% missing values (53909 rows missing).
NoOfURLRedirect has 48.06% missing values (53986 rows missing).
NoOfObfuscatedChar has 47.59% missing values (53460 rows missing).
HasObfuscation has 46.76% missing values (52518 rows missing).
HasPasswordField has 47.49% missing values (53344 rows missing).
NoOfSelfRedirect has 47.67% missing values (53549 rows missing).
LetterRatioInURL has 46.87% missing values (52644 rows missing).
ObfuscationRatio has 46.04% missing values (51714 rows missing).
NoOfLettersInURL has 45.09% missing values (50650 rows missing).
SpacialCharRatioInURL has 44.69% missing values (50197 rows missing).


In [42]:
# Missing values on categorical columns
print("Missing categorical data on training set: ")
missing_vals(categorical_columns, train_df)

Missing categorical data on training set: 
TLD has 32.31% missing values (36286 rows missing).


#### Validation Data

In [43]:
# Missing values on numerical columns
print("Missing numerical data on validation set: ")
missing_vals(numerical_columns,validate_set)

Missing numerical data on validation set: 
LineOfCode has 49.54% missing values (13910 rows missing).
NoOfExternalRef has 49.43% missing values (13881 rows missing).
LargestLineLength has 48.39% missing values (13589 rows missing).
NoOfCSS has 48.38% missing values (13585 rows missing).
HasSocialNet has 48.17% missing values (13526 rows missing).
HasCopyrightInfo has 47.85% missing values (13436 rows missing).
NoOfURLRedirect has 47.71% missing values (13398 rows missing).
NoOfObfuscatedChar has 47.5% missing values (13338 rows missing).
HasObfuscation has 47.01% missing values (13202 rows missing).
HasPasswordField has 46.97% missing values (13191 rows missing).
NoOfSelfRedirect has 46.89% missing values (13166 rows missing).
LetterRatioInURL has 46.66% missing values (13102 rows missing).
ObfuscationRatio has 45.88% missing values (12884 rows missing).
NoOfLettersInURL has 45.18% missing values (12688 rows missing).
SpacialCharRatioInURL has 45.0% missing values (12637 rows missing).

In [44]:
# Missing values on categorical columns
print("Missing categorical data on validation set: ")
missing_vals(categorical_columns, validate_set)

Missing categorical data on validation set: 
TLD has 32.45% missing values (9113 rows missing).


In [45]:
class FeatureImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self, numerical_columns=None, categorical_columns=None):
        self.numerical_columns = numerical_columns
        self.categorical_columns = categorical_columns

    def fit(self, X, y=None):
        # Automatically detect numerical and categorical columns if not provided
        if self.numerical_columns is None:
            self.numerical_columns = X.select_dtypes(include=["float64", "int64"]).columns.tolist()
        if self.categorical_columns is None:
            self.categorical_columns = X.select_dtypes(include=["object"]).columns.tolist()

        # Fit numerical imputer
        if self.numerical_columns:
            self.numerical_imputer = SimpleImputer(strategy="mean")
            self.numerical_imputer.fit(X[self.numerical_columns])

        # Fit categorical imputer
        if self.categorical_columns:
            self.categorical_imputer = SimpleImputer(strategy="most_frequent")
            self.categorical_imputer.fit(X[self.categorical_columns])

        return self

    def transform(self, X):
        X_imputed = X.copy()

        # Impute numerical columns
        if self.numerical_columns:
            X_imputed[self.numerical_columns] = self.numerical_imputer.transform(X_imputed[self.numerical_columns])

        # Impute categorical columns
        if self.categorical_columns:
            X_imputed[self.categorical_columns] = self.categorical_imputer.transform(X_imputed[self.categorical_columns])

        return X_imputed

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [46]:
def remove_outlier(data, columns, exclude_col="label"):

    data = data.copy()
    
    for col in columns:
        if col == exclude_col or col not in data.columns:
            continue

        # Determine IQR for the column
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Calculate cut-off boundaries
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off

        # Clip values to upper and lower limits
        data[col] = data[col].clip(lower=lower, upper=upper)
        
        # Debug information
        print(f"{col}:\n"
              f"  IQR: {IQR}\n"
              f"  Cut-off: {cut_off}\n"
              f"  Lower bound: {lower}\n"
              f"  Upper bound: {upper}\n"
              f"  Values clipped to range [{lower}, {upper}]\n")
    
    return data


In [47]:
# Remove numerical outliers (excluding "label")
train_df = remove_outlier(train_df, numerical_columns, exclude_col="label")
validate_set = remove_outlier(validate_set, numerical_columns, exclude_col="label")

id:
  IQR: 118057.0
  Cut-off: 177085.5
  Lower bound: -118309.0
  Upper bound: 353919.0
  Values clipped to range [-118309.0, 353919.0]

URLLength:
  IQR: 7.0
  Cut-off: 10.5
  Lower bound: 12.5
  Upper bound: 40.5
  Values clipped to range [12.5, 40.5]

DomainLength:
  IQR: 7.0
  Cut-off: 10.5
  Lower bound: 5.5
  Upper bound: 33.5
  Values clipped to range [5.5, 33.5]

IsDomainIP:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 0.0
  Upper bound: 0.0
  Values clipped to range [0.0, 0.0]

CharContinuationRate:
  IQR: 0.07999999999999996
  Cut-off: 0.11999999999999994
  Lower bound: 0.8
  Upper bound: 1.1199999999999999
  Values clipped to range [0.8, 1.1199999999999999]

TLDLegitimateProb:
  IQR: 0.5099802999999999
  Cut-off: 0.7649704499999999
  Lower bound: -0.7520436499999998
  Upper bound: 1.2878775499999997
  Values clipped to range [-0.7520436499999998, 1.2878775499999997]

URLCharProb:
  IQR: 0.008827897500000001
  Cut-off: 0.013241846250000001
  Lower bound: 0.042327996
  Upper boun

NoOfSubDomain:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 1.0
  Upper bound: 1.0
  Values clipped to range [1.0, 1.0]

HasObfuscation:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 0.0
  Upper bound: 0.0
  Values clipped to range [0.0, 0.0]

NoOfObfuscatedChar:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 0.0
  Upper bound: 0.0
  Values clipped to range [0.0, 0.0]

ObfuscationRatio:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 0.0
  Upper bound: 0.0
  Values clipped to range [0.0, 0.0]

NoOfLettersInURL:
  IQR: 6.0
  Cut-off: 9.0
  Lower bound: 1.0
  Upper bound: 25.0
  Values clipped to range [1.0, 25.0]

LetterRatioInURL:
  IQR: 0.14300000000000007
  Cut-off: 0.2145000000000001
  Lower bound: 0.19449999999999987
  Upper bound: 0.7665000000000002
  Values clipped to range [0.19449999999999987, 0.7665000000000002]

NoOfDegitsInURL:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 0.0
  Upper bound: 0.0
  Values clipped to range [0.0, 0.0]

DegitRatioInURL:
  IQR: 0.0
  Cut-off: 0.0
  Lower bound: 0.0
  Upper

In [48]:
print(f'train_set: {train_df.shape}, val_set: {validate_set.shape}')

train_set: (112323, 52), val_set: (28081, 52)


In [49]:
unique_val = train_df['label'].unique()
print(unique_val)

[1 0]


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [50]:
# Duplicate handle
print("Before dropping duplicates:", len(train_df))
train_df.drop_duplicates(inplace=True)
print("After dropping duplicates:", len(train_df))

train_df.reset_index(drop=True, inplace=True)

Before dropping duplicates: 112323
After dropping duplicates: 112323


In [51]:
# Duplicate handle
print("Before dropping duplicates:", len(validate_set))
validate_set.drop_duplicates(inplace=True)
print("After dropping duplicates:", len(validate_set))

validate_set.reset_index(drop=True, inplace=True)

Before dropping duplicates: 28081
After dropping duplicates: 28081


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [52]:
class FeatureCreator(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        if 'TLD' in X.columns:
            n_tld = 10
            self.most_frequent_tld = X['TLD'].value_counts()[:n_tld].index
        else:
            self.most_frequent_tld = []
        
        self.fitted_cols = X.columns
        return self

    def transform(self, X):
        
        X = X.copy()

        if 'IsHTTPS' in X.columns and 'URLLength' in X.columns:
            X['Weighted_IsHTTPS'] = X['IsHTTPS'] * X['URLLength']

        if 'TLD' in X.columns:
            for tld in self.most_frequent_tld:
                X[f'TLD_{tld}'] = (X['TLD'] == tld).astype(int)
            X['TLD'] = X['TLD'].apply(self.recategorize_tld)

        return X

    def recategorize_tld(self, tld):
        return 'Others' if tld not in self.most_frequent_tld else tld

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [53]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [54]:
class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self, method="min_max"):
        self.method = method
        self.scaler = None
        self.numeric_columns = None

    def fit(self, X, y=None):
        
        self.numeric_columns = X.select_dtypes(include=["float64", "int64"]).columns.tolist()

        if self.method == "min_max":
            self.scaler = MinMaxScaler().fit(X[self.numeric_columns])
        elif self.method == "standard":
            self.scaler = StandardScaler().fit(X[self.numeric_columns])
        elif self.method == "robust":
            self.scaler = RobustScaler().fit(X[self.numeric_columns])
        elif self.method == "log":
            self.scaler = None
        else:
            raise ValueError("Invalid scaling method.")

        return self

    def transform(self, X):
        X_transformed = X.copy()

        if self.method in ["min_max", "standard", "robust"]:
            if self.scaler is None:
                raise ValueError("The scaler has not been fitted.")
            X_transformed[self.numeric_columns] = self.scaler.transform(X_transformed[self.numeric_columns])

        elif self.method == "log":
            X_transformed[self.numeric_columns] = np.log1p(X_transformed[self.numeric_columns])

        return X_transformed

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [55]:
# Feature Encoder
class FeatureEncoder(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform (self, X):
        X_encoded = X.copy()

        for col in categorical_columns:
            X_encoded = pd.concat([X_encoded, pd.get_dummies(X_encoded[col], prefix=col)], axis=1)
            X_encoded.drop(col, axis=1, inplace=True)

        return X_encoded

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [56]:
# Oversampling using SMOTE
def balance_data(X, y):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [57]:
pipe = Pipeline([("imputer", FeatureImputer()),
                 ("featurecreator", FeatureCreator()),
                 ("scaler", FeatureScaler()),
                 ("encoder", FeatureEncoder())])

In [58]:
# Apply pipeline transformations

train_df.drop(axis=1,columns=["TLD"])
validate_set.drop(axis=1,columns=["TLD"])

train_processed = pipe.fit_transform(train_df)
validate_processed = pipe.transform(validate_set)


# Extract features and labels
X_train = train_processed.drop(columns=["label"]).values  # Exclude target column
y_train = train_processed["label"].values

X_test = validate_processed.drop(columns=["label"]).values  # Exclude target column
y_test = validate_processed["label"].values

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [59]:
# class KNN:
#     def __init__(self, k=3, metric="euclidean", batch_size=100):
#         '''
#         Inisiliasi awal dari model KNN, memiliki parameter yang menerima input jumlah tetangga (k), 
#         metric untuk mengukur data uji dengan data latih (metric), serta ukuran data yang ingin diuji 
#         sekaligus (batch_size). train_data dan train_labels belum memiliki nilai saat inisialisasi.
#         '''
#         self.k = k
#         self.metric = metric
#         self.batch_size = batch_size
#         self.train_data = None
#         self.train_labels = None
    
#     def fit(self, train_data, train_labels):
#         '''
#         Metode untuk melatih model menggunakan data latih serta hasil dari data latih tersebut,
#         memberikan acuan bagi model untuk mengetahui hasil yang akan didapatkan dari data yang ada.
#         '''
#         self.train_data = np.array(train_data, dtype=np.float32)
#         self.train_labels = np.array(train_labels, dtype=np.int32)
    
#     def calculate_distance(self, test_batch, p=3):
#         '''
#         Menghitung jarak dari data yang ingin diuji dengan data latih yang ada menggunakan metrik yang dipilih
#         oleh pengguna. Menerima parameter seperti partisi data yang ingin diproses (test_batch) serta nilai variabel
#         p yang khusus digunakan untuk metrik minkowski.
#         '''
#         if self.metric == "euclidean":
#             distances = np.sqrt(np.sum((self.train_data[None, :, :] - test_batch[:, None, :]) ** 2, axis=2))
#         elif self.metric == "manhattan":
#             distances = np.sum(np.abs(self.train_data[None, :, :] - test_batch[:, None, :]), axis=2)
#         elif self.metric == "minkowski":
#             distances = np.sum(np.abs(self.train_data[None, :, :] - test_batch[:, None, :]) ** p, axis=2) ** (1 / p)
#         else:
#             raise ValueError(f"Unsupported metric: {self.metric}")
#         return distances
    
#     def predict(self, test_data):
#         '''
#         Memprediksi hasil dari data uji melalui referensi dari data latih dan hasil dari semua data latih tersebut.
#         Metode ini menyimpan hasil dari prediksi ke dalam suatu array yang nantinya dapat digunakan untuk analisis
#         atau evaluasi bagi pengguna.
#         '''
#         test_data = np.asarray(test_data, dtype=np.float32)
#         predictions = np.empty(len(test_data), dtype=self.train_labels.dtype)

#         for start_idx in range(0, len(test_data), self.batch_size):
#             end_idx = min(start_idx + self.batch_size, len(test_data))
#             test_batch = test_data[start_idx:end_idx]

#             distances = self.calculate_distance(test_batch)

#             for i, dist in enumerate(distances):
#                 k_indices = np.argpartition(dist, self.k)[:self.k]
#                 k_labels = self.train_labels[k_indices]
#                 predictions[start_idx + i] = Counter(k_labels).most_common(1)[0][0]

#         return predictions

#     def evaluate(self, test_data, test_labels):
#         '''
#         Metode yang digunakan untuk melihat hasil evaluasi atau kinerja dari model KNN, menggunakan
#         berbagai metrik penilaian seperti akurasi, presisi, recall, f1, matriks confusion, dan laporan
#         klasifikasi.
#         '''
#         predictions = self.predict(test_data)
        
#         accuracy = accuracy_score(test_labels, predictions)
#         precision = precision_score(test_labels, predictions, average="weighted", zero_division=0)
#         recall = recall_score(test_labels, predictions, average="weighted", zero_division=0)
#         f1 = f1_score(test_labels, predictions, average="weighted", zero_division=0)
#         cm = confusion_matrix(test_labels, predictions)
#         report = classification_report(test_labels, predictions, zero_division=0)
        
#         print(f"Accuracy: {accuracy:.2f}")
#         print(f"Precision: {precision:.2f}")
#         print(f"Recall: {recall:.2f}")
#         print(f"F1 Score: {f1:.2f}")
#         print(f"Confusion Matrix:\n{cm}")
#         print("\nClassification Report:\n")
#         print(report)
        
#         return {
#             "accuracy": accuracy,
#             "precision": precision,
#             "recall": recall,
#             "f1": f1,
#             "confusion_matrix": cm,
#             "classification_report": report
#         }

#     def save(self, filename):
#         """
#         Menyimpan model KNN ke dalam sebuah file yang menjadi input pada parameter
#         """
#         with open(filename, 'wb') as f:
#             pickle.dump(self, f)
#         print(f"Model successfully saved to {filename}")

#     @staticmethod
#     def load_model(filename):
#         """
#         Mengambil informasi model KNN dari sebuah file yang menjadi input pada parameter
#         """
#         with open(filename, 'rb') as f:
#             model = pickle.load(f)
#         print(f"Model successfully loaded from {filename}")
#         return model

# knn = KNN(k=5, metric="euclidean", batch_size=100)
# knn.fit(X_train, y_train)
# metrics = knn.evaluate(X_train, y_train)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from concurrent.futures import ThreadPoolExecutor

# '''Inisialisasi model KNN'''
# knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3)

# '''Melatih model KNN'''
# knn.fit(X_train, y_train)

# '''Memprediksi data menggunakan model KNN'''
# def predict_batch(batch):
#     return knn.predict(batch)

# batch_size = 100
# predictions = []

# '''Penggunaan threads untuk efisiensi pengujian'''
# with ThreadPoolExecutor(max_workers=4) as executor:  # Use 4 threads
#     futures = []
#     for start_idx in range(0, len(X_test), batch_size):
#         end_idx = min(start_idx + batch_size, len(X_test))
#         batch = X_test[start_idx:end_idx]
#         futures.append(executor.submit(predict_batch, batch))
    
#     for future in futures:
#         predictions.extend(future.result())

# predictions = np.array(predictions)

# '''Evaluasi hasil model KNN'''
# accuracy = accuracy_score(y_test, predictions)
# precision = precision_score(y_test, predictions, average="weighted", zero_division=0)
# recall = recall_score(y_test, predictions, average="weighted", zero_division=0)
# f1 = f1_score(y_test, predictions, average="weighted", zero_division=0)
# cm = confusion_matrix(y_test, predictions)
# report = classification_report(y_test, predictions, zero_division=0)

# '''Tampilan hasil evaluasi'''
# print(f"Accuracy: {accuracy:.2f}")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1 Score: {f1:.2f}")
# print(f"Confusion Matrix:\n{cm}")
# print("\nClassification Report:\n")
# print(report)

# knn.save("../models/knn.pkl")

## B. Naive Bayes

In [ ]:



class GaussianNaiveBayes:
    def __init__(self):
        # Inisialisasi variabel untuk menyimpan kelas, mean, varians, dan probabilitas awal
        self.classes = None
        self.mean = {}
        self.var = {}
        self.priors = {}

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_samples, n_features = X.shape

        # Menghitung mean, varians, dan probabiltas awal untuk setiap kelas
        for c in self.classes:
            X_c = X[y == c]  
            self.mean[c] = np.mean(X_c, axis=0)  # Mean dari fitur untuk kelas tertentu
            self.var[c] = np.var(X_c, axis=0) + 1e-6  # Varians dari fitur 
            self.priors[c] = X_c.shape[0] / n_samples  # Probabilitas awal untuk kelas

    def _gaussian_pdf(self, class_idx, x):
        # Fungsi distribusi probabilitas Gaussian
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var)) 
        denominator = np.sqrt(2 * np.pi * var) 
        return numerator / denominator

    def predict(self, X):
        predictions = []
        for x in X:
            posteriors = []
            for c in self.classes:
                # Menghitung log prior
                prior = np.log(self.priors[c])
                # Menghitung log likelihood
                conditional = np.sum(np.log(self._gaussian_pdf(c, x)))
                # Posterior = prior + likelihood
                posterior = prior + conditional
                posteriors.append(posterior)
            # Memilih kelas dengan posterior tertinggi
            predictions.append(self.classes[np.argmax(posteriors)])
        return np.array(predictions)
    
    def save(self, filename):
        """
        Menyimpan model Naive Bayes ke dalam sebuah file yang menjadi input pada parameter
        """
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        print(f"Model successfully saved to {filename}")

    @staticmethod
    def load_model(filename):
        """
        Mengambil informasi model Naive Bayes dari sebuah file yang menjadi input pada parameter
        """
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        print(f"Model successfully loaded from {filename}")
        return model

# Membuat dataset dengan 2 kelas
X, y = make_classification(n_samples=500, n_features=5, n_classes=2, random_state=42)
X = np.array(X)
y = np.array(y)

# Membagi dataset
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model Gaussian Naive Bayes
gnb = GaussianNaiveBayes.load_model('../models/gnb.pkl')
# gnb = GaussianNaiveBayes()
# gnb.fit(X_train, y_train)

# Melakukan prediksi pada data validasi
y_pred = gnb.predict(X_validate)

# Mengevaluasi performa model
print("Akurasi:", accuracy_score(y_validate, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_validate, y_pred))

gnb.save('../models/gnb.pkl')

Model successfully loaded from ../models/gnb.pkl
Akurasi: 0.92

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92        53
           1       0.91      0.91      0.91        47

    accuracy                           0.92       100
   macro avg       0.92      0.92      0.92       100
weighted avg       0.92      0.92      0.92       100

Model successfully saved to ../models/gnb.pkl


## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [68]:
# Read the test dataset
test_df = pd.read_csv('../data/test.csv')

# Define the pipeline
pipe = Pipeline([
    ("imputer", FeatureImputer()),
    ("featurecreator", FeatureCreator()),
    ("scaler", FeatureScaler()),
    ("encoder", FeatureEncoder())
])

# Verify 'label' exists in the original training set
if "label" not in train_df.columns:
    raise ValueError("The 'label' column is missing from the training dataset.")

# Preprocess the training set
train_processed = pipe.fit_transform(train_df.copy())

# Debugging: Check columns after preprocessing
print("Columns after pipeline transformation:", train_processed.columns)

# Remove duplicate columns
train_processed = train_processed.loc[:, ~train_processed.columns.duplicated()]

# Debugging: Ensure 'label' exists
if "label" not in train_processed.columns:
    raise ValueError("'label' column is missing after pipeline transformation.")

# Split features and labels
X_train = train_processed.drop(columns=["label"]).values
y_train = train_processed["label"].values

# Train the KNN model
# knn = KNN(k=5, metric="euclidean", batch_size=50)
# knn.fit(X_train, y_train)

# Preprocess the test set
test_df['label'] = ""
test_processed = pipe.transform(test_df)

# Debugging: Ensure 'id' exists in test set
print("Columns in test_processed:", test_processed.columns)

# Remove duplicate columns from the test set
test_processed = test_processed.loc[:, ~test_processed.columns.duplicated()]

# Extract features for prediction
X_test = test_processed.drop(columns=["id"]).values

# Predict on the test set
# test_predictions = knn.predict(X_test)
test_predictions = gnb.predict(X_test)

# Create submission file
submission = test_processed[["id"]].copy()
submission["label"] = test_predictions
submission.to_csv("submission.csv", index=False)

print("Submission file created: submission.csv")


Columns after pipeline transformation: Index(['id', 'URLLength', 'DomainLength', 'IsDomainIP', 'CharContinuationRate',
       'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain',
       'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio',
       'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL',
       'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
       'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon',
       'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect',
       'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit',
       'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields',
       'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo',
       'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef',
       'NoOfExternalRef', 'label', 'Weigh

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: ''

# 5. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`